# 🎨 SDXL + LogoRedmond LoRA API Server

This notebook runs **SDXL Base + LogoRedmond LoRA V2** as an API server.

**Features:**
- ✅ SDXL Base 1.0 (highest quality)
- ✅ LogoRedmond LoRA V2 for professional logos
- ✅ API Server with ngrok
- ✅ Connects to your local Streamlit app

---

In [ ]:
#@title 1️⃣ Install Dependencies
%pip install -q diffusers transformers accelerate torch flask flask-cors pyngrok pillow safetensors
print("✅ Dependencies installed!")

In [ ]:
#@title 2️⃣ Load SDXL Base + LogoRedmond LoRA
import torch
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler
import gc

gc.collect()
torch.cuda.empty_cache()

print("🔄 Loading SDXL Base 1.0...")

# Load SDXL Base
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

# Fast scheduler
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

print("🔄 Loading LogoRedmond LoRA V2...")

# Load LogoRedmond LoRA
pipe.load_lora_weights(
    "artificialguybr/LogoRedmond-LogoLoraForSDXL-V2",
    weight_name="LogoRedmondV2-Logo-LogoRedmAF.safetensors"
)

# Move to GPU
pipe = pipe.to("cuda")

print("\n✅ SDXL + LogoRedmond Ready!")
print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
#@title 3️⃣ Setup ngrok
#@markdown ### Get your FREE ngrok token:
#@markdown 1. Go to https://ngrok.com/ and sign up (free)
#@markdown 2. Go to https://dashboard.ngrok.com/get-started/your-authtoken
#@markdown 3. Copy **ONLY the token** (the long string, NOT the command)

NGROK_TOKEN = "" #@param {type:"string"}

from pyngrok import ngrok

if NGROK_TOKEN and not NGROK_TOKEN.startswith("ngrok"):
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok configured!")
elif NGROK_TOKEN.startswith("ngrok"):
    print("❌ ERROR: Paste ONLY the token, not the full command!")
else:
    print("⚠️ No ngrok token! Get one free at https://ngrok.com/")

In [ ]:
#@title 4️⃣ Start API Server 🚀

from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from io import BytesIO
import threading
import time

PORT = 5001
LORA_TRIGGER = "LogoRedmAF"

app = Flask(__name__)
CORS(app)

def build_prompt(description, style="modern"):
    quality = "vector art, clean lines, sharp edges, high contrast, professional, centered, white background"
    
    styles = {
        "modern": "sleek, geometric, minimalist, contemporary",
        "minimal": "simple, single icon, negative space, ultra clean",
        "vintage": "retro badge, classic, heritage, textured",
        "playful": "mascot, colorful, friendly, cute, rounded",
        "corporate": "professional, trustworthy, symmetrical, shield",
        "tech": "futuristic, circuit, digital, cyber, gradient",
        "organic": "nature, leaf, flowing, eco, natural"
    }
    
    style_kw = styles.get(style.lower(), styles["modern"])
    positive = f"{LORA_TRIGGER}, {description} logo, {style_kw}, {quality}"
    negative = "blurry, low quality, distorted, ugly, text, watermark, signature, photo, realistic, 3d render"
    
    return positive, negative

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "model": "sdxl-base+logoredmond-v2"})

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.json
        description = data.get("prompt", "tech startup")
        style = data.get("style", "modern")
        num_steps = data.get("num_steps", 25)
        guidance = data.get("guidance_scale", 7.5)
        width = data.get("width", 1024)
        height = data.get("height", 1024)
        lora_scale = data.get("lora_scale", 0.9)
        
        positive, negative = build_prompt(description, style)
        
        print(f"📨 Request: {description} ({style})")
        
        image = pipe(
            prompt=positive,
            negative_prompt=negative,
            num_inference_steps=num_steps,
            guidance_scale=guidance,
            width=width,
            height=height,
            cross_attention_kwargs={"scale": lora_scale}
        ).images[0]
        
        buffer = BytesIO()
        image.save(buffer, format="PNG")
        img_base64 = base64.b64encode(buffer.getvalue()).decode()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        print("✅ Logo generated!")
        return jsonify({"success": True, "image": img_base64})
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({"success": False, "error": str(e)})

def run_server():
    app.run(port=PORT, host='0.0.0.0')

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
time.sleep(3)

public_url = ngrok.connect(PORT)
api_url = public_url.public_url

print("\n" + "="*60)
print("🌐 YOUR API URL (Copy this!):")
print(f"\n   {api_url}")
print("\n" + "="*60)
print("\n📋 Paste this URL in your local Streamlit app!")

In [ ]:
#@title 5️⃣ Test API
import requests

print(f"Testing: {api_url}/health")
response = requests.get(f"{api_url}/health")
print(f"Health: {response.json()}")

print("\n🎨 Testing logo generation (~30s)...")
response = requests.post(
    f"{api_url}/generate",
    json={"prompt": "modern tech startup", "style": "tech"},
    timeout=120
)
data = response.json()
if data.get("success"):
    print("✅ Test passed!")
    from PIL import Image
    import io
    img_data = base64.b64decode(data["image"])
    img = Image.open(io.BytesIO(img_data))
    display(img)
else:
    print(f"❌ Error: {data.get('error')}")

---

## ✅ Server Running!

On your local PC:
```bash
cd d:\ai-logo-generator
streamlit run app.py
```

Paste the ngrok URL in the sidebar.